In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd

pd.set_option("max_row", 1500)

<IPython.core.display.Javascript object>

In [3]:
import numpy as np

<IPython.core.display.Javascript object>

In [4]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

<IPython.core.display.Javascript object>

In [5]:
market_basket = pd.read_csv("Market Basket Analysis/market_basket.csv")

<IPython.core.display.Javascript object>

In [6]:
market_basket

,sentence#,word,pos,tag
0,0.0,Preheat,VB,U-Action
1,0.0,oven,NN,U-Utensil
2,0.0,to,IN,O
3,0.0,425,CD,O
4,0.0,degrees,NNS,O
...,...,...,...,...
6804,507.0,an,DT,O
6805,507.0,hour,NN,O
6806,507.0,at,IN,O
6807,507.0,least,JJS,O


<IPython.core.display.Javascript object>

In [7]:
market_basket = market_basket.drop(["pos"], axis=1)

<IPython.core.display.Javascript object>

In [8]:
market_basket

,sentence#,word,tag
0,0.0,Preheat,U-Action
1,0.0,oven,U-Utensil
2,0.0,to,O
3,0.0,425,O
4,0.0,degrees,O
...,...,...,...
6804,507.0,an,O
6805,507.0,hour,O
6806,507.0,at,O
6807,507.0,least,O


<IPython.core.display.Javascript object>

In [9]:
# convert all BILU-Ingredients into Ingredients and combine the words into one
market_basket["tag"].value_counts()

O               4979
U-Action         685
U-Ingredient     499
U-Utensil        122
B-Ingredient     121
L-Ingredient     120
B-Utensil        119
L-Utensil        119
I-Utensil         32
I-Ingredient       7
B-Action           3
L-Action           2
I-Action           1
Name: tag, dtype: int64

<IPython.core.display.Javascript object>

In [10]:
market_basket_consolidated = pd.DataFrame({"sentence#": [], "word": [], "tag": []})
ingredient_word = ""
action_word = ""

for i in range(len(market_basket)):

    # U-Ingredient
    if market_basket.iloc[i]["tag"] == "U-Ingredient":
        market_basket_consolidated = market_basket_consolidated.append(
            pd.DataFrame(
                {
                    "sentence#": [market_basket.iloc[i]["sentence#"]],
                    "word": [market_basket.iloc[i]["word"]],
                    "tag": ["Ingredient"],
                }
            )
        )

    # B-Ingredient
    if market_basket.iloc[i]["tag"] == "B-Ingredient":
        ingredient_word = market_basket.iloc[i]["word"]

    # I-Ingredient, will always be followed by B-Ingredient
    if market_basket.iloc[i]["tag"] == "I-Ingredient":
        ingredient_word = ingredient_word + " " + market_basket.iloc[i]["word"]

    # L-Ingredient, will always be followed by B-Ingredient
    if market_basket.iloc[i]["tag"] == "L-Ingredient":
        ingredient_word = ingredient_word + " " + market_basket.iloc[i]["word"]
        market_basket_consolidated = market_basket_consolidated.append(
            pd.DataFrame(
                {
                    "sentence#": [market_basket.iloc[i]["sentence#"]],
                    "word": [ingredient_word],
                    "tag": ["Ingredient"],
                }
            )
        )

        # reset ingredient_word
        ingredient_word = ""

    # U-Action
    if market_basket.iloc[i]["tag"] == "U-Action":
        market_basket_consolidated = market_basket_consolidated.append(
            pd.DataFrame(
                {
                    "sentence#": [market_basket.iloc[i]["sentence#"]],
                    "word": [market_basket.iloc[i]["word"]],
                    "tag": ["Action"],
                }
            )
        )

    # B-Action
    if market_basket.iloc[i]["tag"] == "B-Action":
        action_word = market_basket.iloc[i]["word"]

    # I-Ingredient, will always be followed by B-Ingredient
    if market_basket.iloc[i]["tag"] == "I-Action":
        action_word = action_word + " " + market_basket.iloc[i]["word"]

    # L-Ingredient, will always be followed by B-Ingredient
    if market_basket.iloc[i]["tag"] == "L-Action":
        action_word = action_word + " " + market_basket.iloc[i]["word"]
        market_basket_consolidated = market_basket_consolidated.append(
            pd.DataFrame(
                {
                    "sentence#": [market_basket.iloc[i]["sentence#"]],
                    "word": [action_word],
                    "tag": ["Action"],
                }
            )
        )

        # reset action_word
        action_word = ""


market_basket_consolidated = market_basket_consolidated.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [11]:
# lemmatize word column, lowercase word column
market_basket_consolidated["word"] = market_basket_consolidated["word"].apply(
    lambda x: (lemmatizer.lemmatize(x)).lower()
)

<IPython.core.display.Javascript object>

In [12]:
# whatever is the ingredient find -1(recent) Action and form association
rules = pd.DataFrame({"Ingredient": [], "Action": []})

for sentence_number, group_df in market_basket_consolidated.groupby("sentence#"):

    associated_action = ""

    # iterate through each row in group
    for i in range(len(group_df)):

        # keep track of the last Action seen
        if group_df.iloc[i]["tag"] == "Action":
            associated_action = group_df.iloc[i]["word"]

        # once we find an ingredient, then create (ingredient, action) pair
        elif group_df.iloc[i]["tag"] == "Ingredient":
            # if action not found then dont create pair
            if associated_action != "":
                rules = rules.append(
                    pd.DataFrame(
                        {
                            "Ingredient": [group_df.iloc[i]["word"]],
                            "Action": [associated_action],
                        }
                    )
                )
rules.reset_index(drop=True, inplace=True)

<IPython.core.display.Javascript object>

In [13]:
rules

,Ingredient,Action
0,sausage,cut
1,egg,whisk
2,milk,whisk
3,sausage,spoon
4,cheese,sprinkle
5,egg,pour
6,sausage,pour
7,cheese,pour
8,egg,bake
9,ground beef,brown


<IPython.core.display.Javascript object>

# Market Basket Analysis

In [24]:
len(market_basket[market_basket["word"] == "sausage"])

4

<IPython.core.display.Javascript object>

In [14]:
rules["Ingredient Count"] = rules.groupby(["Ingredient"]).transform(np.size)

<IPython.core.display.Javascript object>

In [15]:
rules

,Ingredient,Action,Ingredient Count
0,sausage,cut,4
1,egg,whisk,19
2,milk,whisk,7
3,sausage,spoon,4
4,cheese,sprinkle,13
5,egg,pour,19
6,sausage,pour,4
7,cheese,pour,13
8,egg,bake,19
9,ground beef,brown,3


<IPython.core.display.Javascript object>

In [16]:
rules["Ingredient Action Count"] = rules.groupby(["Ingredient", "Action"]).transform(
    np.size
)

<IPython.core.display.Javascript object>

In [31]:
rules["Confidence Percentage"] = (
    np.round(rules["Ingredient Action Count"] / rules["Ingredient Count"], 2) * 100
)

<IPython.core.display.Javascript object>

In [32]:
rules

,Ingredient,Action,Ingredient Count,Ingredient Action Count,Confidence Percentage
0,sausage,cut,4,1,25.0
1,egg,whisk,19,2,11.0
2,milk,whisk,7,1,14.0
3,sausage,spoon,4,1,25.0
4,cheese,sprinkle,13,1,8.0
5,egg,pour,19,2,11.0
6,sausage,pour,4,1,25.0
7,cheese,pour,13,1,8.0
8,egg,bake,19,1,5.0
9,ground beef,brown,3,2,67.0


<IPython.core.display.Javascript object>

In [19]:
results = rules.copy(deep=True)

<IPython.core.display.Javascript object>

In [20]:
results.drop(["Ingredient Count", "Ingredient Action Count"], axis=1, inplace=True)

<IPython.core.display.Javascript object>

In [21]:
# remove duplicate rows
results.drop_duplicates(inplace=True)

# reset the index
results.reset_index(drop=True, inplace=True)

<IPython.core.display.Javascript object>

In [39]:
present = pd.DataFrame()

for index, group in results.groupby("Ingredient"):
    present = present.append(
        # top 5
        group.sort_values("Confidence Percentage", ascending=False).head(5)
    )

present = present.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [40]:
present

,Ingredient,Action,Confidence Percentage
0,angel hair pasta,cook,100.0
1,apple juice,saute,100.0
2,applewood chips,us,100.0
3,avocado,serve,33.0
4,avocado,add,33.0
5,avocado,mash,33.0
6,bacon,lay,100.0
7,baking powder,sift,50.0
8,baking powder,mix,25.0
9,baking powder,combine,25.0


<IPython.core.display.Javascript object>